<a href="https://colab.research.google.com/github/Audrey-Stuart0/Optimization-Project/blob/main/FantasyFootballOptimizationProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing all necessary libraries and connecting to my google drive so I can access data

In [ ]:
# Importing libraries for data visualization, manipulation, and optimization
import seaborn as sns  # Used for creating statistical graphics
import pandas as pd  # Used for data manipulation and analysis (DataFrames)
import pyomo.environ as pe  # Used for optimization modeling
import pyomo.environ as pyo  # this is clearly reduntant but I used different naming conventions on accident so it became necessary or else code would break
from pyomo.opt import SolverFactory  # Used for solving optimization problems
import matplotlib.pyplot as plt  # Used for creating visualizations
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Binary  # Components for defining optimization models
import numpy as np  # Used for numerical computing (arrays, math functions)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading in the data

In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks")

# Read CSV data
w17_data = pd.read_csv('NFLWeek17Stats.csv')

# Read Excel data (multiple sheets)
season_data = pd.read_excel('NFL2024SeasonStats.xlsx', sheet_name=['PassStats', 'RushStats', 'RcvStats', 'DefStats'])

# Read another CSV
salaries_data = pd.read_csv('DKSalaries.csv')

###Before going any further lets take a quick look at what data we will be using for this project

# Data Explanation

## DK Salaries Excel File
**Description:**  
Contains players' salary and stat information (only one tab).

**Data points:**
- **Position**
- **Name + ID**
- **Name**
- **ID**
- **Roster Position**
- **Salary**
- **Game Info**
- **TeamAbbrev**
- **AvgPointsPerGame**

---

## NFL2024SeasonStats Excel File
**Description:**  
Contains information on all game stats for players. This file includes 4 tabs: **PassStats**, **RushStats**, **RcvStats**, and **DefStats**.

### PassStats Tab
**Data points:**
- **Player**
- **Age**
- **Team**
- **Pos** (player position)
- **G** – Games Played
- **GS** – Games Started
- **Cmp** – Completions (number of completed passes)
- **Att** – Pass Attempts
- **Cmp%** – Completion Percentage
- **Yds** – Passing Yards (total yards gained through completed passes)
- **TD** – Touchdowns (total passing touchdowns)
- **TD%** – Touchdown Percentage (TDs per pass attempt)
- **Int** – Interceptions (total interceptions thrown)
- **Int%** – Interception Percentage (INTs per pass attempt)
- **1D** – First Downs (passes resulting in a first down)
- **Succ%** – Success Rate (percentage of successful plays)
- **Lng** – Longest Completed Pass (in yards)
- **Y/A** – Yards per Attempt
- **AY/A** – Adjusted Yards per Attempt (accounts for touchdowns and interceptions)
- **Y/C** – Yards per Completion
- **Y/G** – Yards per Game
- **Rate** – Passer Rating
- **QBR** – Total Quarterback Rating (ESPN’s metric)
- **Sk** – Sacks
- **Yds** – Sack Yards Lost
- **Sk%** – Sack Percentage
- **NY/A** – Net Yards per Attempt (accounts for sack yards lost)
- **ANY/A** – Adjusted Net Yards per Attempt

### RushStats Tab
**Data points:**
- **Player**
- **Age**
- **Team**
- **Pos** (player position)
- **G** – Games Played
- **GS** – Games Started
- **Att** – Rushing Attempts (number of times the player carried the ball)
- **Yds** – Rushing Yards
- **TD** – Rushing Touchdowns
- **1D** – First Downs
- **Succ%** – Success Rate (percentage of successful rush attempts)
- **Lng** – Longest Rush
- **Y/A** – Yards per Attempt
- **Y/G** – Yards per Game
- **A/G** – Attempts per Game
- **Fmb** – Fumbles

### RcvStats Tab
**Data points:**
- **RK** – Rank (based on performance)
- **Player**
- **Age**
- **Team**
- **Pos** (player position)
- **G** – Games Played
- **GS** – Games Started
- **Tgt** – Targets (number of times the player was targeted with a pass)
- **Rec** – Receptions (number of caught passes)
- **Yds** – Receiving Yards
- **Y/R** – Yards per Reception
- **TD** – Receiving Touchdowns
- **1D** – First Downs
- **Succ%** – Success Rate (percentage of receptions leading to a successful play)
- **Lng** – Longest Reception
- **R/G** – Receptions per Game
- **Y/G** – Yards per Game
- **Ctch%** – Catch Percentage
- **Y/Tgt** – Yards per Target
- **Fmb** – Fumbles

### DefStats Tab
**Data points:**
- **Tm** – Team Name
- **G** – Games Played
- **PA** – Points Allowed
- **Yds** – Total Offensive Yards
- **Ply** – Total Offensive Plays
- **Y/P** – Yards per Play
- **TO** – Turnovers
- **FL** – Fumbles Lost
- **1stD** – First Downs (by the offense)
- **Pass Cmp** – Pass Completions
- **Pass Att** – Pass Attempts
- **Pass Yds** – Passing Yards
- **Pass TD** – Passing Touchdowns
- **Pass Int** – Passing Interceptions
- **Pass NY/A** – Net Yards per Pass Attempt
- **Pass 1stD** – First Downs by Passing
- **Rush Att** – Rushing Attempts
- **Rush Yds** – Rushing Yards
- **Rush TD** – Rushing Touchdowns
- **Rush 1stD** – First Downs from Rushing
- **Pen** – Penalties
- **Pen Yds** – Penalty Yards
- **Pen 1stD** – First Downs Given Up Due to Penalties
- **Score Drv Pct** – Scoring Drive Percentage
- **TO Drv Pct** – Turnover Drive Percentage
- **Exp Pts Contributed** – Expected Points Contributed

---

## NFLWeek17Stats Excel File
**Description:**  
Contains detailed game stats for Week 17.

**Data points:**
- **Player**
- **Tm**
- **Pass Cmp** – Pass Completions
- **Pass Att** – Pass Attempts
- **Pass Yds** – Passing Yards
- **Pass TD** – Passing Touchdowns
- **Pass Int** – Interceptions Thrown
- **Pass Sack** – Number of Times Sacked
- **Pass Sack Yds** – Total Sack Yards Lost
- **Pass Long Complete** – Longest Completed Pass
- **Rush Att** – Rushing Attempts (*Note: Listed as "Ruch Att" in the file*)
- **Rush Yds** – Rushing Yards
- **Rush TD** – Rushing Touchdowns
- **Rush Lng** – Longest Rush
- **Rcv Tgt** – Receiving Targets
- **Rec** – Receptions
- **Rcv Yds** – Receiving Yards
- **Rcv TD** – Receiving Touchdowns
- **Rcv Lng** – Longest Reception
- **Fmb** – Fumbles
- **FL** – Fumbles Lost
- **2Pt Conversions** – Successful Two-Point Conversions
- **Ttl Pts Allowed** – Total Points Allowed (mostly for defenders)
- **Safety** – Safeties
- **Blocked Kicks** – Field Goals or Punts Blocked
- **Sack** – Sacks on Opposing Quarterback
- **Interception** – Interceptions Made
- **Fumble Recovery** – Fumbles Recovered
- **Punt/Kickoff/FG Return for TD** – Return Touchdowns
- **Interception Return TD** – Touchdowns on Interception Returns
- **Fumble Recovery TD** – Touchdowns on Fumble Recoveries
- **Blocked Punt or FG Return TD** – Touchdowns on Blocked Plays

###Necissary additional column for later steps:
Before merging defense_season and salaries_defense I need to make it so they have matching columns. Currently I have a Tm column in defense_season that holds the entire name of the team including the city and then I have a Name column that contains the name of the team without the city in the salaries defense data. In order to make this data merge I will reimport the data so it is not cleaned. Then I will split the Name portion out of the defense_salaries Tm column so I can map it to the Name column in salaries defense

In [ ]:
#exrtract the DefStats sheet
defense_season = season_data['DefStats']

# Extract the 'Tm' column
tm_column = defense_season['Tm']

# Create a new 'Team' column by extracting the last word of each 'Tm' entry
defense_season['Team'] = tm_column.str.split().str[-1]

# Display the updated DataFrame (optional)
#defense_season


Now lets clean the data so its easier to use and data merges properly

In [ ]:
# Remove suffixes from 'Name' column in salaries_data
#Name column because thats what I merge on
salaries_data['Name'] = salaries_data['Name'].str.replace(r'\s*\(.*\)', '', regex=True).str.strip()

# Remove suffixes from 'Player' column in Season_data (assuming it's a DataFrame, not a dictionary)
#(player column because thats the column I use to merge)
for sheet_name, df in season_data.items():
    if 'Player' in df.columns:
        season_data[sheet_name]['Player'] = df['Player'].str.replace(r'\s*\(.*\)', '', regex=True).str.strip()


In [ ]:
import re #a module that allows for text manipulation and pattern matching

def clean_column(df, column_name): #defines a clean function column that takes two parameters, a dataframe and a column
  df[column_name] = df[column_name].astype(str).str.lower() #double checks everything is a string and then converts everything to lowercase
  df[column_name] = df[column_name].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x))#Uses re.sub() to remove anything that is NOT a letter or number,
    # ^ → Negates the match (everything except letters and numbers)., a-zA-Z0-9 → Keeps only letters (both uppercase & lowercase) and numbers.
  df[column_name] = df[column_name].replace('nan', 0)#replaces all nan values with 0s
  return df#Returns the modified DataFrame after cleaning


# useds the clean_column function to clean Player columns in season_data
for sheet_name in ['PassStats', 'RushStats', 'RcvStats']:
  season_data[sheet_name] = clean_column(season_data[sheet_name], 'Player')

# Clean Name column in salaries_data
salaries_data = clean_column(salaries_data, 'Name')

# uses the clean_colum function to clean Tm column in DefStats
season_data['DefStats'] = clean_column(season_data['DefStats'], 'Tm')


I had to clean the Player column specifically because I was losing certain players when I merged the salary data and the w17_data because of mitch match naming conventions

In [ ]:
#stupidly made a second clean_column function. I initially thought I would need a second one because i thought i would need to clean each
#column in a different way but it turned out that I needed the same functionalitly for both so this one ended up being redundant
def clean_column(df, column_name):
    df[column_name] = df[column_name].astype(str).str.lower()
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x))
    df[column_name] = df[column_name].replace('nan', 0)
    return df

# Clean Player column in w17_data
w17_data = clean_column(w17_data, 'Player')

In [ ]:
# Extract individual sheets from the Excel file
#In case I need to access them seperatly or in case I want to add specific sheets to other data frames this makes it easier for later use
pass_stats = season_data['PassStats']
rush_stats = season_data['RushStats']
rcv_stats = season_data['RcvStats']
def_stats = season_data['DefStats']

##Splitting week 17 data into offense and defense

I decided to split my data into offense and defense and heres why:

Different Scoring Systems
- Offensive and defensive players contribute to fantasy points using entirely different criteria. Offense focuses on yards, touchdowns, and receptions, while defense earns points through sacks, turnovers, and points allowed. Separating them ensures accurate optimization based on relevant performance metrics.

Distinct Player Selection Strategies

- The selection criteria for offensive and defensive players differ. Offensive players are chosen based on their ability to accumulate yards and touchdowns, whereas defenses are evaluated by their ability to limit opponent scoring and force turnovers. Splitting the data allows for tailored decision-making for each group.

Flexibility in Lineup Construction

- In fantasy football, offensive players have multiple positions (QB, RB, WR, TE, FLEX), while defense is selected as a single unit (DST). Keeping offense and defense separate ensures the optimization model correctly handles positional constraints and avoids misallocations.

In [ ]:
offense_week17 = w17_data.iloc[0:322].copy() #split at 322 because thats where defense data starts
#offense_week17

# Extract and print the defensive rows for the week data
defense_week17 = w17_data.iloc[322:353].copy()
#defense_week17

Some of the team names in the week 17 data did not match the naming conventions of the salary data even though the teams were the same. In order to fix this I directly mapped the non matching teams so I would not lose any data.

In [ ]:
#change some of the Tm names so they will map better to salary data later on
# Standardize team abbreviations in the 'Tm' column of defense_week17
team_abbreviation_mapping = {
    "GNB": "GB",
    "SFO": "SF",
    "NOR": "NO",
    "LVR": "LV",
    "TAM": "TB",
    "NEW": "NE"
}

defense_week17["Tm"] = defense_week17["Tm"].replace(team_abbreviation_mapping)


###Seperating Salary into offense and defense

In order to easily combine salary data to week 17 data and later on season data I split salary into offense and defense because thats how I sorted my week 17 and season data.

In [ ]:
# Separate defense (DST) and offense players from salaries_data
salaries_defense = salaries_data[salaries_data["Roster Position"] == "DST"].copy()
salaries_offense = salaries_data[salaries_data["Roster Position"] != "DST"].copy()

Adding salary data to the defense and offense week 17 data

In [ ]:
offense_week17 = pd.merge(salaries_offense, offense_week17, left_on='Name', right_on='Player', how='inner')
#offense_week17

In [ ]:
# Merge defense_season and salaries_defense on 'tm' and 'name' respectively
defense_week17 = defense_week17.merge(salaries_defense, left_on='Tm', right_on='TeamAbbrev', how='inner')
#defense_week17

###Yay! Our data is all ready for question 1!

So what is question 1?

Question 1: The first problem will involve selecting the optimal lineup based on how the players
performed during their actual games (i.e., the “cheat code” if you had it prior to selecting
your lineup). What selection of NFL players would have created the best possible entry for
this DraftKings competition?

###Calculate Offense and defense fantasy points

In order to begin answering question one we need to create a function that properly calculates offense fantasy points and defense fantasy points. Below I created functions that calculate and add the calculated fantasy points to the offense_week17 and then the defense_week17. I followed every instruction given for the calculations and stated what each piece of the code is doing for clarity.

In [ ]:
def calculate_offense_fantasy_points(row): #this function takes one parameter (row)
    """Calculate fantasy points for an offensive player."""
    offense_fantasy_points = 0

    # Passing stats
    offense_fantasy_points += 4 * row.get('Pass TD', 0)  # +4 pts per passing TD
    offense_fantasy_points += 0.04 * row.get('Pass Yds', 0)  # +0.04 pts per passing yard
    if row.get('Pass Yds', 0) >= 300:  # +3 pts for 300+ passing yards
        offense_fantasy_points += 3
    offense_fantasy_points += -1 * row.get('Pass Int', 0)  # -1 pt per interception

    # Rushing stats
    offense_fantasy_points += 6 * row.get('Rush TD', 0)  # +6 pts per rushing TD
    offense_fantasy_points += 0.1 * row.get('Rush Yds', 0)  # +0.1 pts per rushing yard
    if row.get('Rush Yds', 0) >= 100:  # +3 pts for 100+ rushing yards
        offense_fantasy_points += 3

    # Receiving stats
    offense_fantasy_points += 6 * row.get('Rcv TD', 0)  # +6 pts per receiving TD
    offense_fantasy_points += 0.1 * row.get('Rcv Yds', 0)  # +0.1 pts per receiving yard
    if row.get('Rcv Yds', 0) >= 100:  # +3 pts for 100+ receiving yards
        offense_fantasy_points += 3
    offense_fantasy_points += 1 * row.get('Rec', 0)  # +1 pt per reception

    # Miscellaneous
    offense_fantasy_points += -1 * row.get('Fmb', 0)  # -1 pt per fumble lost
    offense_fantasy_points += 2 * row.get('2pt Conversions', 0)  # +2 pts per 2-point conversion
    offense_fantasy_points += 6 * row.get('Fumble Recovery TD', 0)  # +6 pts per offensive fumble recovery TD

    # Special Teams / Miscellaneous
    offense_fantasy_points += 6 * row.get('Punt/Kickoff/FG Return for TD', 0)

    return offense_fantasy_points

# Apply the function to create a new 'FantasyPoints' column
offense_week17['FantasyPoints'] = offense_week17.apply(calculate_offense_fantasy_points, axis=1)
#axis = 1 means it inserts the data row by row


Now for Defense

In [ ]:
def calculate_defense_fantasy_points(row): #this function takes one parameter (row)
    """Calculate fantasy points for a defensive team."""
    defense_fantasy_points = 0

    # Start the points at 0 to add to it
    defense_fantasy_points = 0

    # Defensive stats
    defense_fantasy_points += 1 * row['Sack']  # +1 pt per sack
    defense_fantasy_points += 2 * row['Interception']  # +2 pts per interception
    defense_fantasy_points += 2 * row['Fumble Recovery']  # +2 pts per fumble recovery
    defense_fantasy_points += 6 * row['Punt/Kickoff/FG Return for TD']  # +6 pts per return TD
    defense_fantasy_points += 6 * row['Interception Return TD']  # +6 pts per interception return TD
    defense_fantasy_points += 6 * row['Fumble Recovery TD']  # +6 pts per fumble recovery TD
    defense_fantasy_points += 6 * row['Blocked Punt or FG Return TD']  # +6 pts per blocked punt/FG return TD
    defense_fantasy_points += 2 * row['Safety']  # +2 pts per safety
    defense_fantasy_points += 2 * row['Blocked Kicks']  # +2 pts per blocked kick
    defense_fantasy_points += 2 * row['2Pt Conversions']  # +2 pts per 2pt conversion return

    # Defensive Points Allowed
    points_allowed = row.get('Ttl Pts Allowed', 0)  #Retrieves the total points allowed by the defense.
                                                    #Uses .get('Ttl Pts Allowed', 0) to default to 0 if the column is missing.

    if points_allowed == 0:
        defense_fantasy_points += 10
    elif 1 <= points_allowed <= 6:
        defense_fantasy_points += 7
    elif 7 <= points_allowed <= 13:
        defense_fantasy_points += 4
    elif 14 <= points_allowed <= 20:
        defense_fantasy_points += 1
    elif 21 <= points_allowed <= 27:
        defense_fantasy_points += 0  # No points change
    elif 28 <= points_allowed <= 34:
        defense_fantasy_points -= 1
    elif points_allowed >= 35:
        defense_fantasy_points -= 4

    return defense_fantasy_points


    # Apply the function to each row in nfl_df
defense_week17['Fantasy Score'] = defense_week17.apply(calculate_defense_fantasy_points, axis=1)
#axis = 1 means it inserts the data row by row

Just checking if the columns added properly:

In [ ]:
#defense_week17

In [ ]:
#offense_week17

###Filtering by player position

In order to "easily" calculate position-specific constraints and logic later in the optimization model I split the players up by position. In addition the goal of this optimization model is to select or not select each player with the goal of creating the highest scoring team. In order for the model to select or not select each player for each position it is easiest for the players to be split up by position.

In [ ]:
# Filter players by position
qbs = offense_week17[offense_week17['Position'] == 'QB'] #retrieves the position column in the offense_week17 data frame and then takes the rows where the data == QB
rbs = offense_week17[offense_week17['Position'] == 'RB']
wrs = offense_week17[offense_week17['Position'] == 'WR']
tes = offense_week17[offense_week17['Position'] == 'TE']
flex_players = pd.concat([rbs, wrs, tes]) # Combine RB, WR, and TE for FLEX
dsts = defense_week17

Just a quick check to make sure we have data for each position before trying to run the model

In [ ]:
print(f"Available QBs: {len(qbs)}")
print(f"Available RBs: {len(rbs)}")
print(f"Available WRs: {len(wrs)}")
print(f"Available TEs: {len(tes)}")
print(f"Available FLEX options: {len(flex_players)}")
print(f"Available DSTs: {len(dsts)}")

Available QBs: 35
Available RBs: 70
Available WRs: 102
Available TEs: 53
Available FLEX options: 225
Available DSTs: 26


Initializing the model

In [ ]:
# Create an optimization model
model = pe.ConcreteModel()

Creating binary decision variables for each player. The model will end up using these decision variables to either select or not select each player it tests. Allowing it to eventually select the best players for the highest scoring team

In [ ]:
# Define decision variables for each player
model.qb_vars = pe.Var(qbs.index, within=pe.Binary)
model.rb_vars = pe.Var(rbs.index, within=pe.Binary)
model.wr_vars = pe.Var(wrs.index, within=pe.Binary)
model.te_vars = pe.Var(tes.index, within=pe.Binary)
model.flex_vars = pe.Var(flex_players.index, within=pe.Binary)
model.dst_vars = pe.Var(dsts.index, within=pe.Binary)

###Objective function

This piece of code tells the model what it should try to maximize which in this case is FantasyPoints for offense and Fantasy Score for defense.

How does my objective function work:
- In my Pyomo model, I defined the objective function using the Objective class and set the goal to pe.maximize, since we want to maximize total fantasy points."
-"Each term in this function corresponds to a different player position, ensuring that we sum up the fantasy points for all selected players."

Breaking Down the Components
"Each line of the objective function represents a different position group:"
- Loops through all QBs (i in qbs.index).
- model.qb_vars[i] is a binary decision variable (1 if the QB is selected, 0 if not).
- qbs.loc[i, 'FantasyPoints'] retrieves the fantasy points for each QB.
- If a QB is selected, their fantasy points contribute to the total.

this same logic is used for all the players


Since DST is team-based, it uses 'Fantasy Score' instead of 'FantasyPoints'.
Ensures only one DST is selected, contributing to the total score.

how does it optimize:
- "Since each model.qb_vars[i], model.rb_vars[i], etc., is a binary variable (0 or 1), the objective function only sums up the fantasy points for players that are actually selected in the final lineup. The optimizer finds the best combination of players that maximizes this sum while still satisfying constraints like salary cap and roster requirements."


In [ ]:
# Define the objective function to maximize total fantasy points
def objective_rule(model): #defining function which takes the model object which contains the cdecision variables as input
    return sum(model.qb_vars[i] * qbs.loc[i, 'FantasyPoints'] for i in qbs.index) + \ #multiplies whatever quarterback was chosen by their fantasy points so the model can mazimize total fantasy points for the team
    #for i in qbs.index → Loop through every QB (each i is an index in qbs.index).
    #qbs.index represents all quarterbacks' indices in the DataFrame.
    #The loop iterates over each QB's index (i) and applies the expression inside sum()
           sum(model.rb_vars[i] * rbs.loc[i, 'FantasyPoints'] for i in rbs.index) + \
           sum(model.wr_vars[i] * wrs.loc[i, 'FantasyPoints'] for i in wrs.index) + \
           sum(model.te_vars[i] * tes.loc[i, 'FantasyPoints'] for i in tes.index) + \
           sum(model.flex_vars[i] * flex_players.loc[i, 'FantasyPoints'] for i in flex_players.index) + \
           sum(model.dst_vars[i] * dsts.loc[i, 'Fantasy Score'] for i in dsts.index)

model.objective = pe.Objective(rule=objective_rule, sense=pe.maximize)
#model.objective:  It stores the objective function, which tells the solver what to optimize.
#pe.Objectiv : creates the objective function in Pyomo
#rule=objective_rule → Specifies what the objective function should be.
#sense=pe.maximize → Specifies whether we want to maximize or minimize

###Constraints

Based off what we were given:

The fantasy team must have nine “players” (a team defense/special teams counts as one player).
2. The salary of your eight players may not exceed $50,000.
3. You need players from at least two different games.
4. Your 9-player fantasy team roster should include the following number of players from each of
these positions:
a. 1 Quarterback (QB)
b. 2 Running Backs (RB)
c. 3 Wide Receivers (WR)
d. 1 Tight End (TE)
e. 1 FLEXible position (a RB, WR, or TE)
f. 1 Team’s Defense/Special Teams (DST

Note: I originally had the constraints in different code chunks but it was difficult to run and rerun the constraints when they were seperated so thats why I ended up combining them.

In [ ]:
# Constraint: Maximum of 9 players in the team
#Defines a constraint function that takes the optimization model as input.
#Pyomo calls this function when defining constraints.
# then it loops through all quarterbacks and since the model.qb_vars is already
#a binary variable it becomes 1 if selected 0 if not it sums the number of quartebacks chosen so it
#can later follow the == 9 constraint
def max_players_rule(model):
    return sum(model.qb_vars[i] for i in qbs.index) + \
           sum(model.rb_vars[i] for i in rbs.index) + \
           sum(model.wr_vars[i] for i in wrs.index) + \
           sum(model.te_vars[i] for i in tes.index) + \
           sum(model.flex_vars[i] for i in flex_players.index) + \
           sum(model.dst_vars[i] for i in dsts.index) == 9

model.max_players_constraint = pe.Constraint(rule=max_players_rule) #defines the constraint and calls the max_players_rule funtion


# Constraint: Exactly one defense team
def exactly_one_dst_rule(model):
    return sum(model.dst_vars[i] for i in dsts.index) == 1

model.exactly_one_dst_constraint = pe.Constraint(rule=exactly_one_dst_rule) #defines constraint and calls the exactly_one_dst_rule


# Constraint: Salary cap
#1️ Define the Function:
#The function salary_cap_rule(model) is defined to enforce the salary cap constraint in the optimization model.
#2️ Calculate Total Salary for QBs:
#Loops through all quarterbacks (QBs) and multiplies each binary selection variable (0 or 1) by the corresponding salary from the dataset.
#3️ Calculate Total Salary for RBs, WRs, and TEs:
#Similar to QBs, it loops through running backs (RBs), wide receivers (WRs), and tight ends (TEs), summing up salaries for the selected players.
#4️ Include FLEX Position Players:
#Since the FLEX position can be an RB, WR, or TE, this term ensures that the salary of the selected FLEX player is counted.
#5️ Include Defense/Special Teams (DST):
#The salary for the selected defense team is also added to the total.
#6️ Enforce the Salary Cap:

The final sum of all selected players' salaries is constrained to be ≤ $50,000, ensuring that the optimizer does not exceed the fantasy league’s salary limit.
def salary_cap_rule(model):
    return sum(model.qb_vars[i] * qbs.loc[i, 'Salary'] for i in qbs.index) + \
           sum(model.rb_vars[i] * rbs.loc[i, 'Salary'] for i in rbs.index) + \
           sum(model.wr_vars[i] * wrs.loc[i, 'Salary'] for i in wrs.index) + \
           sum(model.te_vars[i] * tes.loc[i, 'Salary'] for i in tes.index) + \
           sum(model.flex_vars[i] * flex_players.loc[i, 'Salary'] for i in flex_players.index)+ \
           sum(model.dst_vars[i] * dsts.loc[i, 'Salary'] for i in dsts.index) <= 50000

model.salary_cap_constraint = pe.Constraint(rule=salary_cap_rule) #defines the constraint and calls the salary_cap rule


# Constraint: Exactly one QB
def exactly_one_qb_rule(model):
    return sum(model.qb_vars[i] for i in qbs.index) == 1 #looks through the available qbs stored int he qbs data frame and then uses the
                                                          #binary decision variable to select or not select that player

model.exactly_one_qb_constraint = pe.Constraint(rule=exactly_one_qb_rule)

# Constraint: Exactly two RBs
def exactly_two_rbs_rule(model):
    return sum(model.rb_vars[i] for i in rbs.index) == 2

model.exactly_two_rbs_constraint = pe.Constraint(rule=exactly_two_rbs_rule)

# Constraint: Exactly three WRs
def exactly_three_wrs_rule(model):
    return sum(model.wr_vars[i] for i in wrs.index) == 3

model.exactly_three_wrs_constraint = pe.Constraint(rule=exactly_three_wrs_rule)

# Constraint: Exactly one TE
def exactly_one_te_rule(model):
    return sum(model.te_vars[i] for i in tes.index) == 1

model.exactly_one_te_constraint = pe.Constraint(rule=exactly_one_te_rule)

# Constraint: Exactly one FLEX (RB, WR, or TE)
def exactly_one_flex_rule(model):
    return sum(model.flex_vars[i] for i in flex_players.index) == 1

model.exactly_one_flex_constraint = pe.Constraint(rule=exactly_one_flex_rule)


# Constraint: Each player can only be selected once (modified for FLEX)
def player_uniqueness_rule(model):
    all_players = set(qbs['Player']).union(set(rbs['Player']), set(wrs['Player']), set(tes['Player'])) #This line creates a set of all unique player names across different position groups (QB, RB, WR, TE) to ensure player uniqueness in the lineup.
    #This gathers all unique player names from QBs, RBs, WRs, and TEs into a set.
    #The set() function removes duplicate names.                                                      # this was the best way I could figure out how to make sure a flex player is not chosen again

    for player in all_players: #iterates over every unique player in the dataset with a goal of seeing if they have been selected more than once
        # Count selections across all positions, including FLEX
          #Each sum(...) loops through a specific position group (QB, RB, WR, TE, FLEX).
          #It checks if the current player is selected (1 or 0) in that position.
          #The total selected_count is the number of times a player has been selected.
              #qbs.loc[i, 'Player'] gets the name of the player at index i in the QBs dataset.
              #player is the current player name from all_players that we're checking in the loop.
              #if qbs.loc[i, 'Player'] == player keeps only rows where the player name matches.
        selected_count = sum(model.qb_vars[i] for i in qbs.index if qbs.loc[i, 'Player'] == player) + \
                         sum(model.rb_vars[i] for i in rbs.index if rbs.loc[i, 'Player'] == player) + \
                         sum(model.wr_vars[i] for i in wrs.index if wrs.loc[i, 'Player'] == player) + \
                         sum(model.te_vars[i] for i in tes.index if tes.loc[i, 'Player'] == player) + \
                         sum(model.flex_vars[i] for i in flex_players.index if flex_players.loc[i, 'Player'] == player)

        # Constraint for each player to be selected at most once
        model.add_component(f'player_uniqueness_constraint_{player}', pe.Constraint(expr=selected_count <= 1))

    # Defense
    for player in set(dsts['Player']):
        model.add_component(f'player_uniqueness_constraint_dst_{player}', pe.Constraint(expr=sum(model.dst_vars[i] for i in dsts.index if dsts.loc[i, 'Player'] == player) <= 1))

    return Constraint.Feasible

model.player_uniqueness_constraint = pe.Constraint(rule=player_uniqueness_rule)


# Constraint: Players from at least 2 different games
def at_least_two_games_rule(model):
    games = set(pd.concat([qbs['Game Info'], rbs['Game Info'], wrs['Game Info'], tes['Game Info'], flex_players['Game Info'], dsts['Game Info']]))
    # Create binary variables for each game and add to the model *before* the loop
    model.game_vars = pe.Var(games, within=pe.Binary)

    # Link game variables to player selections
    for game in games: #iterates over all unique games
        # For each position group, check if a player is in the current game and add constraints
            #🔹 Loops through all unique games in the dataset.
            #🔹 Checks if any players from a given game are selected in each position group.
            #🔹 Creates constraints that link player selection to game selection.
        if game in qbs['Game Info'].values: #looks at the Game Info column for qbs and if game appears in this column it means at least one qb is playing in tis game this just prevents unnecessary constraint creation for games with no players in that position
            model.add_component(f'game_constraint_qb_{game}', #adds constraint and links it to players games
                                  pe.Constraint(expr=sum(model.qb_vars[i] for i in qbs.index if qbs.loc[i, 'Game Info'] == game) <= len(qbs) * model.game_vars[game]))

        if game in rbs['Game Info'].values:
            model.add_component(f'game_constraint_rb_{game}',
                                  pe.Constraint(expr=sum(model.rb_vars[i] for i in rbs.index if rbs.loc[i, 'Game Info'] == game) <= len(rbs) * model.game_vars[game]))

        if game in wrs['Game Info'].values:
            model.add_component(f'game_constraint_wr_{game}',
                                  pe.Constraint(expr=sum(model.wr_vars[i] for i in wrs.index if wrs.loc[i, 'Game Info'] == game) <= len(wrs) * model.game_vars[game]))

        if game in tes['Game Info'].values:
            model.add_component(f'game_constraint_te_{game}',
                                  pe.Constraint(expr=sum(model.te_vars[i] for i in tes.index if tes.loc[i, 'Game Info'] == game) <= len(tes) * model.game_vars[game]))

        if game in flex_players['Game Info'].values:
            model.add_component(f'game_constraint_flex_{game}',
                                  pe.Constraint(expr=sum(model.flex_vars[i] for i in flex_players.index if flex_players.loc[i, 'Game Info'] == game) <= len(flex_players) * model.game_vars[game]))

        if game in dsts['Game Info'].values:
            model.add_component(f'game_constraint_dst_{game}',
                                  pe.Constraint(expr=sum(model.dst_vars[i] for i in dsts.index if dsts.loc[i, 'Game Info'] == game) <= len(dsts) * model.game_vars[game]))

    # Constraint: Ensure at least 2 different games are selected
    return sum(model.game_vars[game] for game in games) >= 2  #These ensure that if a player from a game is selected, the corresponding model.game_vars[game] binary variable is set to 1.

model.at_least_two_games_constraint = pe.Constraint(rule=at_least_two_games_rule)


In [ ]:
# Solve the model
solver = pe.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)  # Store the results object

Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 9e+03]
  Cost   [8e-02, 4e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 5e+04]
Presolving model
315 rows, 511 cols, 2493 nonzeros  0s
232 rows, 460 cols, 1283 nonzeros  0s
Objective function is integral with scale 50

Solving MIP model with:
   232 rows
   460 cols (460 binary, 0 integer, 0 implied int., 0 continuous)
   1283 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0 

###Extracting and Displaying the Selected Optimal Lineup

What This Code Does:
This code extracts the selected players from the optimization model, calculates the total salary and fantasy points of the chosen lineup, and prints the final selection.



In [ ]:
selected_players = [] #→ Stores the selected lineup as a list of tuples (Player Name, Position, Salary, Fantasy Points)
total_salary = 0 #starting at 0 so we can add to it
max_fantasy_points = 0 #starting at 0 so we can add to it

# Iterate through the decision variables and extract selected players
#Each loop iterates through a specific position group (QB, RB, WR, TE, FLEX, DST) and checks which players were selected (value == 1)
for i in qbs.index:
    if model.qb_vars[i].value == 1: #loops through all the quarterbacks in qbs.index and checks if the optimizer selected the player
        selected_players.append((qbs.loc[i, 'Name'], 'QB', qbs.loc[i, 'Salary'], qbs.loc[i, 'FantasyPoints'])) #adds whatever players where selected to the tuple selected_players
        total_salary += qbs.loc[i, 'Salary']  #adds the corresponding salaries of the players selected to the total salary
        max_fantasy_points += qbs.loc[i, 'FantasyPoints'] #adds the corresponding fantasy points of the players selected to the total points
for i in rbs.index:
    if model.rb_vars[i].value == 1:
        selected_players.append((rbs.loc[i, 'Name'], 'RB', rbs.loc[i, 'Salary'], rbs.loc[i, 'FantasyPoints']))
        total_salary += rbs.loc[i, 'Salary']
        max_fantasy_points += rbs.loc[i, 'FantasyPoints']
for i in wrs.index:
    if model.wr_vars[i].value == 1:
        selected_players.append((wrs.loc[i, 'Name'], 'WR', wrs.loc[i, 'Salary'], wrs.loc[i, 'FantasyPoints']))
        total_salary += wrs.loc[i, 'Salary']
        max_fantasy_points += wrs.loc[i, 'FantasyPoints']
for i in tes.index:
    if model.te_vars[i].value == 1:
        selected_players.append((tes.loc[i, 'Name'], 'TE', tes.loc[i, 'Salary'], tes.loc[i, 'FantasyPoints']))
        total_salary += tes.loc[i, 'Salary']
        max_fantasy_points += tes.loc[i, 'FantasyPoints']
for i in flex_players.index:
    if model.flex_vars[i].value == 1:
        selected_players.append((flex_players.loc[i, 'Name'], flex_players.loc[i, 'Position'], flex_players.loc[i, 'Salary'], flex_players.loc[i, 'FantasyPoints']))
        total_salary += flex_players.loc[i, 'Salary']
        max_fantasy_points += flex_players.loc[i, 'FantasyPoints']

for i in dsts.index:
    if model.dst_vars[i].value == 1:
        selected_players.append((dsts.loc[i, 'Name'], 'DST', dsts.loc[i, 'Salary'], dsts.loc[i, 'Fantasy Score']))
        total_salary += dsts.loc[i, 'Salary']
        max_fantasy_points += dsts.loc[i, 'Fantasy Score']


# Output the results
print("Selected Players:")
for player, position, salary, fantasy_points in selected_players:
    print(f"{player} ({position}): Salary - ${salary}, Fantasy Points - {fantasy_points}")

print(f"\nMax Fantasy Points: {max_fantasy_points}")
print(f"Total Salary: ${total_salary}")


Selected Players:
drewlock (QB): Salary - $4900, Fantasy Points - 37.36
jonathantaylor (RB): Salary - $7900, Fantasy Points - 30.6
buckyirving (RB): Salary - $6700, Fantasy Points - 26.0
teehiggins (WR): Salary - $6900, Fantasy Points - 44.1
devontasmith (WR): Salary - $5800, Fantasy Points - 33.0
rickypearsall (WR): Salary - $4200, Fantasy Points - 31.700000000000003
zachertz (TE): Salary - $3800, Fantasy Points - 25.2
maliknabers (WR): Salary - $6600, Fantasy Points - 39.1
eagles (DST): Salary - $3100, Fantasy Points - 22

Max Fantasy Points: 289.06
Total Salary: $49900


#Yay! Time for Question 2

The second problem will involve selecting a lineup based on historical information.
Assuming you didn’t have the “cheat code”, what would be your strategy in choosing
players, and how it would outperform the naïve strategy of just selecting the players based
upon their average daily points? For example, perhaps you think older players will play
worse than their averages because this is at the end of the season and they are likely to
fatigue more easily. Or perhaps you think you should select players playing at home,
because you think they will better than their averages at home than they would on the road
(especially at Christmas!) Maybe you think players will do better if they are playing against a
bad team, or worse if they are playing against a good team? Maybe you want to pick people
who are on the same team, because if one player is throwing a touchdown pass, his
teammate might be receiving the pass? If you don’t know anything about American
football, use Google (or your favorite AI)!

### Data Set up

Lets start by getting our defense_season data and our offense_season data and merging them with salary data. Very similar to what we did with the week 17 data

In [ ]:
# Merge all offensive stats
offense_season = pd.concat([pass_stats, rush_stats, rcv_stats], axis=0, ignore_index=True)

In [ ]:
# Merge offense_season and salaries_offense on 'Player' and 'Name' respectively
offense_season = pd.merge(offense_season, salaries_offense, left_on='Player', right_on='Name', how='inner')

In [ ]:
offense_season

,Player,Age,Team,Pos,G,GS,Cmp,Att,Cmp%,Yds,...,Y/Tgt,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,joeburrow,28,CIN,QB,17,17,460.0,652.0,70.6,4918,...,NaN,QB,Joe Burrow (37128257),joeburrow,37128257,QB,7200,DEN@CIN 12/28/2024 04:30PM ET,CIN,23.59
1,jaredgoff,30,DET,QB,17,17,390.0,539.0,72.4,4629,...,NaN,QB,Jared Goff (37128259),jaredgoff,37128259,QB,6800,DET@SF 12/30/2024 08:15PM ET,DET,20.96
2,bakermayfield,29,TAM,QB,17,17,407.0,570.0,71.4,4500,...,NaN,QB,Baker Mayfield (37128258),bakermayfield,37128258,QB,6900,CAR@TB 12/29/2024 01:00PM ET,TB,22.77
3,genosmith,34,SEA,QB,17,17,407.0,578.0,70.4,4320,...,NaN,QB,Geno Smith (37128274),genosmith,37128274,QB,5500,SEA@CHI 12/26/2024 08:15PM ET,SEA,17.42
4,samdarnold,27,MIN,QB,17,17,361.0,545.0,66.2,4319,...,NaN,QB,Sam Darnold (37128260),samdarnold,37128260,QB,6400,GB@MIN 12/29/2024 04:25PM ET,MIN,19.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,malikwillis,25,GNB,QB,7,2,NaN,NaN,NaN,0,...,NaN,QB,Malik Willis (37128338),malikwillis,37128338,QB,4000,GB@MIN 12/29/2024 04:25PM ET,GB,8.01
1038,jameiswinston,30,CLE,QB,12,7,NaN,NaN,NaN,0,...,NaN,QB,Jameis Winston (37128286),jameiswinston,37128286,QB,4800,MIA@CLE 12/29/2024 04:05PM ET,CLE,12.84
1039,bryceyoung,23,CAR,QB,14,12,NaN,NaN,NaN,0,...,NaN,QB,Bryce Young (37128272),bryceyoung,37128272,QB,5600,CAR@TB 12/29/2024 01:00PM ET,CAR,12.72
1040,darekeyoung,25,SEA,WR,14,0,NaN,NaN,NaN,0,...,0.0,WR,Dareke Young (37129188),darekeyoung,37129188,WR/FLEX,3000,SEA@CHI 12/26/2024 08:15PM ET,SEA,0.00


In order to merge defense_season with salaries_defense I need to lowercase the defense_season data or else nothing will map properly and I will get no data

In [ ]:
#lowercase all the data in the Team column in the defense_season data
defense_season['Team'] = defense_season['Team'].str.lower()

In [ ]:
# merge defense_season on Team column and salaries_defense on Name column
defense_season = pd.merge(defense_season, salaries_defense, left_on='Team', right_on='Name', how='inner')

Yay! We now have season data combined with salary data for both offense and defense!

Now lets add the fantasy points to offense

Note(There is missing data in the defense sheet so we will use a different metric to see how well our model is working for the defense data)

We first need to make sure any Nan values are 0 or else some calculations will not work

In [ ]:
# Replace NaN values with 0 in the offense_season DataFrame
offense_season.fillna(0, inplace=True)
defense_season.fillna(0, inplace=True)

Lets start by calcuting offense fantasy points. I will be using this as the testing variable to see how well my new constraints perform for offense players.

In [ ]:
def calculate_offense_fantasy_points(row):
    """Calculate fantasy points for an offensive player."""
    offense_fantasy_points = 0

    # Passing stats
    offense_fantasy_points += 4 * row.get('Pass TD', 0)  # +4 pts per passing TD
    offense_fantasy_points += 0.04 * row.get('Pass Yds', 0)  # +0.04 pts per passing yard
    if row.get('Pass Yds', 0) >= 300:  # +3 pts for 300+ passing yards
        offense_fantasy_points += 3
    offense_fantasy_points += -1 * row.get('Pass Int', 0)  # -1 pt per interception

    # Rushing stats
    offense_fantasy_points += 6 * row.get('Rush TD', 0)  # +6 pts per rushing TD
    offense_fantasy_points += 0.1 * row.get('Rush Yds', 0)  # +0.1 pts per rushing yard
    if row.get('Rush Yds', 0) >= 100:  # +3 pts for 100+ rushing yards
        offense_fantasy_points += 3

    # Receiving stats
    offense_fantasy_points += 6 * row.get('Rcv TD', 0)  # +6 pts per receiving TD
    offense_fantasy_points += 0.1 * row.get('Rcv Yds', 0)  # +0.1 pts per receiving yard
    if row.get('Rcv Yds', 0) >= 100:  # +3 pts for 100+ receiving yards
        offense_fantasy_points += 3
    offense_fantasy_points += 1 * row.get('Rec', 0)  # +1 pt per reception

    # Miscellaneous
    offense_fantasy_points += -1 * row.get('Fmb', 0)  # -1 pt per fumble lost
    offense_fantasy_points += 2 * row.get('2pt Conversions', 0)  # +2 pts per 2-point conversion
    offense_fantasy_points += 6 * row.get('Fumble Recovery TD', 0)  # +6 pts per offensive fumble recovery TD

    # Special Teams / Miscellaneous
    offense_fantasy_points += 6 * row.get('Punt/Kickoff/FG Return for TD', 0)

    return offense_fantasy_points

# Apply the function to create a new 'FantasyPoints' column
offense_season['FantasyPoints'] = offense_season.apply(calculate_offense_fantasy_points, axis=1)

#What is my Strategy

Lets try a few different constraints and test them based on fantasy points for offense and AvgPointsPerGame for defense. The best performing constraints will be put into my actual model and I will use that model to pick my team from the week 17 data.

I am going to start by adding a home vs away column so we can see if offense or defense perform better when they are in their home stadium

In [ ]:
#extracts the home and way teams from the game info column by splitting the data in the string into seperate teams at the @
#it them compares each teams abbreviation (TeamAbbrev) and sees if it matches the home team column
offense_season[['home_team', 'away_team']] = offense_season['Game Info'].str.split('@', expand=True)
offense_season['home'] = offense_season['TeamAbbrev'] == offense_season['home_team']

In [ ]:
same as above
defense_season[['home_team', 'away_team']] = defense_season['Game Info'].str.split('@', expand=True)
defense_season['Home'] = defense_season['TeamAbbrev'] == defense_season['home_team']

In [ ]:
#a function that analyzes performanse of offense players by using the Fantasy Score column in offense and then compares to the home column and finds out if players perform better at their home field or away

def analyze_home_away_performance(df, position):

    # Filter for the specified position
        #explanation:
        #df['Position'] extracts the 'Position' column from the DataFrame.
        #== position compares each row's value in the 'Position' column to the variable position
        #basically decides if the row should be included or not
    position_df = df[df['Position'] == position]

    # Calculate the mean fantasy points at home and away
        #walk through:
        #position_df['home'] == True → Filters the dataset for only home games.
        #.loc[..., 'FantasyPoints'] → Selects the 'FantasyPoints' column for those games.
        #.mean() → Computes the average fantasy points for home games.
    home_performance = position_df.loc[position_df['home'] == True, 'FantasyPoints'].mean()
    away_performance = position_df.loc[position_df['home'] == False, 'FantasyPoints'].mean()

    print(f"Analysis for {position}:")
    print(f"Average Fantasy Points at Home: {home_performance:.2f}")
    print(f"Average Fantasy Points Away: {away_performance:.2f}")

    if home_performance > away_performance:
        print(f"{position} players tend to perform better at home.")
    elif away_performance > home_performance:
        print(f"{position} players tend to perform better away.")
    else:
        print(f"No significant difference in performance observed between home and away games for {position} players.")

    return home_performance, away_performance

# Example usage (assuming 'offense_season' DataFrame is available):
#I probably just asked chat to print out the function so I didnt have to type this outgood to be efficient with little things
analyze_home_away_performance(offense_season, 'QB')
analyze_home_away_performance(offense_season, 'RB')
analyze_home_away_performance(offense_season, 'WR')
analyze_home_away_performance(offense_season, 'TE')


Analysis for QB:
Average Fantasy Points at Home: -2.91
Average Fantasy Points Away: -2.31
QB players tend to perform better away.
Analysis for RB:
Average Fantasy Points at Home: 7.73
Average Fantasy Points Away: 6.60
RB players tend to perform better at home.
Analysis for WR:
Average Fantasy Points at Home: 13.36
Average Fantasy Points Away: 13.68
WR players tend to perform better away.
Analysis for TE:
Average Fantasy Points at Home: 10.81
Average Fantasy Points Away: 11.14
TE players tend to perform better away.


(10.81, 11.138613861386139)

Based off the results I will not make a constraint of offense players needing to be home players for the week 17 game

In [ ]:
def analyze_defense_performance(defense_data):

    # Calculate the mean AvgPointsPerGame at home and away
             #walk through:
        #defense_data['Home'] == True → Filters the dataset for only home games.
        #.loc[..., 'AvgPointsPerGame'] → Selects the 'AvgPointsPerGame' column for those games.
        #.mean() → Computes the average fantasy points for home games.
    home_performance = defense_data.loc[defense_data['Home'] == True, 'AvgPointsPerGame'].mean()
    away_performance = defense_data.loc[defense_data['Home'] == False, 'AvgPointsPerGame'].mean()

    print(f"Analysis for Defense:")
    print(f"Average AvgPointsPerGame at Home: {home_performance:.2f}")
    print(f"Average AvgPointsPerGame Away: {away_performance:.2f}")

    if home_performance > away_performance:
        print(f"Defense players tend to perform better at home.")
    elif away_performance > home_performance:
        print(f"Defense players tend to perform better away.")
    else:
        print(f"No significant difference in performance observed between home and away games for defense players.")

    return home_performance, away_performance

# Example usage (assuming 'defense_season' DataFrame is available):
analyze_defense_performance(defense_season)


Analysis for Defense:
Average AvgPointsPerGame at Home: 6.38
Average AvgPointsPerGame Away: 6.18
Defense players tend to perform better at home.


(6.385, 6.175714285714286)

It is true based off the data above that defense teams do better when they perform at their home team. However, since the difference is so slight I do not think it would be smart to constrain the defense team to only being the home team.

My next test I am going to do to find another constraint is find out if certain aged players perform better than other aged players

In [ ]:
#function to analyze performance of offense players based off age using Fantasy points to see how well they are performing
def analyze_player_performance_by_age(df, position): #function takes two parameters dataframe and position

    # Filter for the specified position
   #explanation:
        #df['Position'] extracts the 'Position' column from the DataFrame.
        #== position compares each row's value in the 'Position' column to the variable position
        #basically decides if the row should be included or not
    position_df = df[df['Position'] == position]

    # Group players by age ranges and calculate the average fantasy points
    age_performance = {} #emplty dictionary to store avg fantasy points for the age groups
    for age_start in range(20, 45, 5):  # I didnt see anyone above the age of 45 in my data so this felt like a good age range (and no one below 20)
        age_end = age_start + 5
        age_group = position_df[(position_df['Age'] >= age_start) & (position_df['Age'] < age_end)] #extracts age column from position_df checks if each players age is greater than or equal to age start and checks if each players age is less than age_end and keeps only when condition is true
        avg_fantasy_points = age_group['FantasyPoints'].mean()

        if not pd.isna(avg_fantasy_points): #check for NAN
          age_performance[f"{age_start}-{age_end}"] = avg_fantasy_points


    return age_performance

In [ ]:
# testing for different positions
qb_age_performance = analyze_player_performance_by_age(offense_season, 'QB')
print("QB Age Performance:", qb_age_performance)

rb_age_performance = analyze_player_performance_by_age(offense_season, 'RB')
print("RB Age Performance:", rb_age_performance)

wr_age_performance = analyze_player_performance_by_age(offense_season, 'WR')
print("WR Age Performance:", wr_age_performance)

te_age_performance = analyze_player_performance_by_age(offense_season, 'TE')
print("TE Age Performance:", te_age_performance)

QB Age Performance: {'20-25': -2.727272727272727, '25-30': -2.673076923076923, '30-35': -1.8888888888888888, '35-40': -3.466666666666667, '40-45': -3.3333333333333335}
RB Age Performance: {'20-25': 8.023255813953488, '25-30': 6.627586206896551, '30-35': 8.1}
WR Age Performance: {'20-25': 14.0828025477707, '25-30': 11.989189189189188, '30-35': 16.862068965517242}
TE Age Performance: {'20-25': 10.425925925925926, '25-30': 11.336538461538462, '30-35': 11.292682926829269, '40-45': 0.5}


Based off the output there is not a strong correlation between age and performance so we will not use this constraint

The next condition I want to test is whether or not there is an impact of stacking a QB with WRs or TEs from the same team.

In [ ]:
#a function that determines if there is a positive impact of stacking qb with wr or tes from the same team. use the Fantasy Points column from offense_season

def analyze_stacking_impact(df):

    team_stacking_impact = {}
    for team in df['Team'].unique(): #extracts all unique team names from the team column and loops them seperatly
        team_df = df[df['Team'] == team] #Filters the dataset to include only players from the current team in the loop.
        qb_players = team_df[team_df['Position'] == 'QB'] #extracts all qb from qb_players
        wr_te_players = team_df[team_df['Position'].isin(['WR', 'TE'])] # extracts all WR and TEs because thats what we will stack

        if not qb_players.empty and not wr_te_players.empty:#checks if the stacking is possible (does the team have atleast one qb and one wr or te if not it skips the team)
            stacked_fantasy_points = []
            non_stacked_fantasy_points = []

            # Simulate stacked lineups #the _, is a throaway variable that throwaway variable that lets puthon ignore values that arent used so I dont have to assign variables to the index that arent used
            for _, qb_row in qb_players.iterrows(): #loops all qbs on the team
              for _, wr_te_row in wr_te_players.iterrows(): #Loops through all WRs & TEs on the same team (wr_te_players)
                  stacked_fantasy_points.append(qb_row['FantasyPoints'] + wr_te_row['FantasyPoints']) #adds the fantasy points and puts them in the stacked fantasy points list

            # Simulate non-stacked lineups (using the same players)
            for _, qb_row in qb_players.iterrows():
                non_stacked_fantasy_points.append(qb_row['FantasyPoints'])
            for _, wr_te_row in wr_te_players.iterrows():
                non_stacked_fantasy_points.append(wr_te_row['FantasyPoints'])

            team_stacking_impact[team] = {
                'stacked_avg': np.mean(stacked_fantasy_points) if stacked_fantasy_points else 0,
                'non_stacked_avg': np.mean(non_stacked_fantasy_points) if non_stacked_fantasy_points else 0,
            }

    return team_stacking_impact

# Example usage:
stacking_impact = analyze_stacking_impact(offense_season)

# Print the results
for team, impact in stacking_impact.items():
    print(f"Team: {team}")
    print(f"  Average Stacked Fantasy Points: {impact['stacked_avg']:.2f}")
    print(f"  Average Non-Stacked Fantasy Points: {impact['non_stacked_avg']:.2f}")
    print(f"  Difference: {impact['stacked_avg'] - impact['non_stacked_avg']:.2f}")


Team: CIN
  Average Stacked Fantasy Points: 11.15
  Average Non-Stacked Fantasy Points: 11.85
  Difference: -0.71
Team: DET
  Average Stacked Fantasy Points: 13.80
  Average Non-Stacked Fantasy Points: 11.44
  Difference: 2.36
Team: TAM
  Average Stacked Fantasy Points: 8.89
  Average Non-Stacked Fantasy Points: 9.46
  Difference: -0.57
Team: SEA
  Average Stacked Fantasy Points: 11.71
  Average Non-Stacked Fantasy Points: 10.78
  Difference: 0.94
Team: MIN
  Average Stacked Fantasy Points: 15.80
  Average Non-Stacked Fantasy Points: 12.43
  Difference: 3.37
Team: NYJ
  Average Stacked Fantasy Points: 12.42
  Average Non-Stacked Fantasy Points: 10.48
  Difference: 1.94
Team: LAC
  Average Stacked Fantasy Points: 8.73
  Average Non-Stacked Fantasy Points: 8.00
  Difference: 0.73
Team: SFO
  Average Stacked Fantasy Points: 8.37
  Average Non-Stacked Fantasy Points: 6.83
  Difference: 1.53
Team: ARI
  Average Stacked Fantasy Points: 14.56
  Average Non-Stacked Fantasy Points: 11.95
  Diff

As a whole there is a poitive impact of stacking team members on the same team.I will try to incorporate this into my model

Next I will sort the offense_season data so that we only use the most recent 8 games for each player. This will make our model more accurate because it will base of more recent data. It will probably make our model perform poorly on the old data we have though when we test on the exisiting week 17 data so that has to be kept in mind.

Start by adding a Game data column to the offense_season data so we can sort by most recent games for each player

In [ ]:
def sort_and_filter_offense_data(offense_season):
    """Sorts offense_season data by player and game date, keeping only the 10 most recent games per player."""

    # Extract the date from the 'Game Info' column (last part of the string)
    offense_season['Game Date'] = offense_season['Game Info'].apply(
        lambda x: re.search(r'\d{1,2}/\d{1,2}/\d{4}', x).group() if pd.notna(x) else None #searches for the date in the format:MM/DD/YYYY or M/D/YYYY
    )

    # Convert the extracted 'Game Date' to a datetime format
    offense_season['Game Date'] = pd.to_datetime(offense_season['Game Date'], format='%m/%d/%Y')

# Apply the function
offense_season_filtered = sort_and_filter_offense_data(offense_season)

#offense_season

In [ ]:
def filter_recent_games(offense_season, num_games=8):
    """Filters the offense_season data to keep only the most recent `num_games` for each player."""

    # Drop any rows where 'Game Date' is missing
    offense_season = offense_season.dropna(subset=['Game Date'])

    # Sort by Player and Game Date (most recent first)
    offense_season_sorted = offense_season.sort_values(by=['Player', 'Game Date'], ascending=[True, False])

    # Keep only the most recent `num_games` per player
    offense_season_filtered = offense_season_sorted.groupby('Player').head(num_games)

    return offense_season_filtered

# Apply the function to filter down to 8 games per player
offense_season_filtered = filter_recent_games(offense_season, num_games=8)

# Display the new dataset size
print(f"Original dataset size: {len(offense_season)}")
print(f"Filtered dataset size: {len(offense_season_filtered)}")


Original dataset size: 1042
Filtered dataset size: 1041


Hmmm thats strange? Why is it only removing 1 data point. Lets check what the average amount of games a player plays in is

In [ ]:
# Calculate the average number of games played per player
average_games_per_player = offense_season['Player'].value_counts().mean()

# Display the result
print(f"The average number of games a player played in: {average_games_per_player:.2f}")


The average number of games a player played in: 2.27


It looks like each player only plays in 2 games on average so using a filter for the past 8 games makes no sense.

Ok we tested a few different concepts in the end the constraints I will add to the model are:

- Interception Penalty Constraint
- QB-WR/TE Positive Stacking Constraint
- Consistency-Based Player Selection

I will explain each of these constraints in the constraint section

##Now lets start the model!

Step 1: filter players by poistion

In [ ]:
# Filter players by position
qbs = offense_season[offense_season['Position'] == 'QB']
rbs = offense_season[offense_season['Position'] == 'RB']
wrs = offense_season[offense_season['Position'] == 'WR']
tes = offense_season[offense_season['Position'] == 'TE']
flex_players = pd.concat([rbs, wrs, tes])
flex_players['Position'] = 'FLEX'  # Add a 'Position' column to identify flex players
dsts = defense_season

In [ ]:
print(f"Available QBs: {len(qbs)}")
print(f"Available RBs: {len(rbs)}")
print(f"Available WRs: {len(wrs)}")
print(f"Available TEs: {len(tes)}")
print(f"Available FLEX options: {len(flex_players)}")
print(f"Available DSTs: {len(dsts)}")

Available QBs: 200
Available RBs: 241
Available WRs: 400
Available TEs: 201
Available FLEX options: 842
Available DSTs: 28


##Step 2: Use an Optimization Solver to Select the Best Lineup
Now that I have all players categorized, I will:

- Create decision variables (each player is either selected (1) or not selected (0)).
- Set constraints for positions and salary cap.
- Tell the solver to maximize total fantasy points.
- Find the best lineup!

In [ ]:
# Create an optimization model
model.x = pe.ConcreteModel()

In [ ]:
# Define decision variables for each player
model.x.qb_vars = pe.Var(qbs.index, within=pe.Binary)
model.x.rb_vars = pe.Var(rbs.index, within=pe.Binary)
model.x.wr_vars = pe.Var(wrs.index, within=pe.Binary)
model.x.te_vars = pe.Var(tes.index, within=pe.Binary)
model.x.flex_vars = pe.Var(flex_players.index, within=pe.Binary)
model.x.dst_vars = pe.Var(dsts.index, within=pe.Binary)

In [ ]:
# Define the objective function to maximize total fantasy points
def objective_rule(x):
    return sum(x.qb_vars[i] * qbs.loc[i, 'FantasyPoints'] for i in qbs.index) + \
           sum(x.rb_vars[i] * rbs.loc[i, 'FantasyPoints'] for i in rbs.index) + \
           sum(x.wr_vars[i] * wrs.loc[i, 'FantasyPoints'] for i in wrs.index) + \
           sum(x.te_vars[i] * tes.loc[i, 'FantasyPoints'] for i in tes.index) + \
           sum(x.flex_vars[i] * flex_players.loc[i, 'FantasyPoints'] for i in flex_players.index) + \
           sum(x.dst_vars[i] * dsts.loc[i, 'AvgPointsPerGame'] for i in dsts.index) #We are not using Fantasy points because there is not enough data in the defense

model.x.objective = pe.Objective(rule=objective_rule, sense=pe.maximize)


###Now lets import our given constraints

In [ ]:
# Define constraints for the number of players at each position
model.x.qbs_constraint = pe.Constraint(expr=sum(model.x.qb_vars[i] for i in qbs.index) == 1)
model.x.rbs_constraint = pe.Constraint(expr=sum(model.x.rb_vars[i] for i in rbs.index) == 2)
model.x.wrs_constraint = pe.Constraint(expr=sum(model.x.wr_vars[i] for i in wrs.index) == 3)
model.x.tes_constraint = pe.Constraint(expr=sum(model.x.te_vars[i] for i in tes.index) == 1)
model.x.flex_constraint = pe.Constraint(expr=sum(model.x.flex_vars[i] for i in flex_players.index) == 1)
model.x.dsts_constraint = pe.Constraint(expr=sum(model.x.dst_vars[i] for i in dsts.index) == 1)

#Total players constraint
model.x.total_players = pe.Constraint(expr=sum(model.x.qb_vars[i] for i in qbs.index) +
                                     sum(model.x.rb_vars[i] for i in rbs.index) +
                                     sum(model.x.wr_vars[i] for i in wrs.index) +
                                     sum(model.x.te_vars[i] for i in tes.index) +
                                     sum(model.x.flex_vars[i] for i in flex_players.index) +
                                     sum(model.x.dst_vars[i] for i in dsts.index) == 9)


In [ ]:
# Define the salary cap constraint
model.x.salary_cap = pe.Constraint(expr=sum(model.x.qb_vars[i] * qbs.loc[i, 'Salary'] for i in qbs.index) +
                                     sum(model.x.rb_vars[i] * rbs.loc[i, 'Salary'] for i in rbs.index) +
                                     sum(model.x.wr_vars[i] * wrs.loc[i, 'Salary'] for i in wrs.index) +
                                     sum(model.x.te_vars[i] * tes.loc[i, 'Salary'] for i in tes.index) +
                                     sum(model.x.flex_vars[i] * flex_players.loc[i, 'Salary'] for i in flex_players.index) +
                                     sum(model.x.dst_vars[i] * dsts.loc[i, 'Salary'] for i in dsts.index) <= 50000)


In [ ]:
# Create a set of unique game IDs from the 'Game Info' column
games = set()
for index, row in offense_season.iterrows():
    games.add(row['Game Info'])
for index, row in defense_season.iterrows():
    games.add(row['Game Info'])
game_list = list(games)
game_index = range(len(game_list))
game_map = dict(zip(game_list, game_index))


# Constraint: At least 2 different games represented
model.x.game_constraint = pe.Constraint(expr=sum(model.x.qb_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in qbs.index) +
                                        sum(model.x.rb_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in rbs.index) +
                                        sum(model.x.wr_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in wrs.index) +
                                        sum(model.x.te_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in tes.index) +
                                        sum(model.x.flex_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in flex_players.index) +
                                        sum(model.x.dst_vars[i] * game_map[defense_season.loc[i, 'Game Info']] for i in dsts.index) >= 2 )


##Now I am adding my given constraints:

### Interception Penalty Constraint

Add a constraint to my model that doesnt allow you to select the quarter backs with the highest 10% of interception rates.

Why?
This constraint ensures that the optimization model avoids selecting quarterbacks who are prone to turnovers, as interceptions negatively impact fantasy points. By eliminating the top 10% of quarterbacks with the highest interception rates, the model prioritizes more consistent and reliable players who are less likely to lose points due to costly mistakes. This improves the overall scoring potential of the lineup while reducing risk, leading to a more optimal and stable fantasy team selection.

In [ ]:
# Calculate interception percentage and adjusted fantasy points
offense_season['Int%'] = offense_season['Int'] / offense_season['Att']   #devides the interceptions by pass attempts to find percentage
offense_season['AdjustedFantasyPoints'] = offense_season['FantasyPoints'] * (1 - (offense_season['Int%'] / 10)) #adjusts the fantasy points of players by reducing the fantasy points based on the interception % (if int% is 5% then reduce fantasy points by 5%)

# Identify the top 10% of QBs by interception rate
top_10_percent_int_qbs = offense_season[offense_season['Position'] == 'QB'].nlargest(int(0.1 * len(offense_season[offense_season['Position'] == 'QB'])), 'Int%')

# Constraint: Exclude QBs in the top 10% of interception rates
model.x.interception_constraint = pe.Constraint(expr=sum(model.x.qb_vars[i] for i in top_10_percent_int_qbs.index) == 0)


###QB-WR/TE Positive Stacking Constraint

This constraint ensures that if a quarterback (QB) is selected, at least one wide receiver (WR) or tight end (TE) from the same team must also be selected in the lineup. It does this by checking each team's available QBs and WR/TEs, and only applying the rule to teams where both positions exist. This promotes stacking, which increases the potential for higher fantasy points by capitalizing on correlated player performances (e.g., a QB throwing a touchdown to their WR or TE).

In [ ]:
# Constraint: If a QB is selected, at least one WR or TE from the same team must also be selected
model.x.qb_wr_te_stacking = pe.ConstraintList() #Creates an empty constraint list (qb_wr_te_stacking) within the model

for team in offense_season['Team'].unique(): #extracts all unique names from the team column in the offense_season data iterates over each team to apply the constraints
    qb_indices = [i for i in qbs.index if qbs.loc[i, 'Team'] == team] #extracts all indices of all gbs on the current team. uses a list combrehension to check if team matches the current team
    wr_te_indices = [i for i in wrs.index if wrs.loc[i, 'Team'] == team] + [i for i in tes.index if tes.loc[i, 'Team'] == team] #finsa all wr and tes of the current team

    # Only apply the constraint if both QBs and WR/TEs exist for this team
    if qb_indices and wr_te_indices: #only loosk at teams where there are atleast 1 wr/te to go with qb for the team
        qb_selected = sum(model.x.qb_vars[i] for i in qb_indices) #counts selected qbs and adds to the sum
        wr_te_selected = sum(model.x.wr_vars[i] for i in wr_te_indices if i in wrs.index) + \
                         sum(model.x.te_vars[i] for i in wr_te_indices if i in tes.index) #Counts the number of selected WRs and TEs for this team.
                                                                                          #Uses if i in wrs.index and if i in tes.index to ensure the correct variables are referenced.

        # Add the constraint
        model.x.qb_wr_te_stacking.add(qb_selected <= wr_te_selected)


###Consistency-Based Player Selection

 Why? Instead of just selecting players with the highest fantasy points, we ensure that at least some players have consistent performance rather than being "boom-or-bust."

 How? Use the Fantasy Points per Game (AvgPointsPerGame) column to ensure that at least 4 players have an average above a set threshold (e.g., 10 points per game).

In [ ]:
# Define a threshold for "consistent" players
consistency_threshold = 10  # Players must average at least 10 fantasy points per game

# Identify players who meet this threshold
consistent_players = offense_season[offense_season['AvgPointsPerGame'] >= consistency_threshold].index.tolist()

In [ ]:
# Constraint: At least 4 selected players must have AvgPointsPerGame >= consistency_threshold
def high_floor_player_selection_rule(x):
    consistent_players_selected = (
        sum(model.x.qb_vars[i] for i in qbs.index if i in consistent_players) + #checks how many players meet the consisten_players threshhold defined above
        sum(model.x.rb_vars[i] for i in rbs.index if i in consistent_players) +
        sum(model.x.wr_vars[i] for i in wrs.index if i in consistent_players) +
        sum(model.x.te_vars[i] for i in tes.index if i in consistent_players) +
        sum(model.x.flex_vars[i] for i in flex_players.index if i in consistent_players)
    )
    return consistent_players_selected >= 4  # Require at least 4 consistent players

model.x.high_floor_player_selection = pe.Constraint(rule=high_floor_player_selection_rule)

In [ ]:
# Solve the model using appsi_highs
opt = SolverFactory('appsi_highs')
results = opt.solve(model.x)

In [ ]:
# Extract the chosen players from the solution
chosen_players = []

for i in qbs.index:
    if model.x.qb_vars[i].value == 1:
        chosen_players.append((qbs.loc[i, 'Player'], 'QB'))

for i in rbs.index:
    if model.x.rb_vars[i].value == 1:
        chosen_players.append((rbs.loc[i, 'Player'], 'RB'))

for i in wrs.index:
    if model.x.wr_vars[i].value == 1:
        chosen_players.append((wrs.loc[i, 'Player'], 'WR'))

for i in tes.index:
    if model.x.te_vars[i].value == 1:
        chosen_players.append((tes.loc[i, 'Player'], 'TE'))

for i in flex_players.index:
    if model.x.flex_vars[i].value == 1:
        chosen_players.append((flex_players.loc[i, 'Player'], flex_players.loc[i, 'Position']))

for i in dsts.index:
    if model.x.dst_vars[i].value == 1:
        chosen_players.append((dsts.loc[i, 'Team'], 'DST'))

# Print the chosen players
for player, position in chosen_players:
    print(f"{player} ({position})")


claytontune (QB)
devonachane (RB)
javontewilliams (RB)
jamarrchase (WR)
maliknabers (WR)
wandalerobinson (WR)
treymcbride (TE)
wandalerobinson (FLEX)
broncos (DST)


In [ ]:
# Access the objective function value (maximum fantasy points + average points scored)
max_fantasy_points = model.x.objective()

print(f"Maximum Projected Fantasy Points: {max_fantasy_points}")

Maximum Projected Fantasy Points: 669.53


##Now lets run this model and these constraints but test it with the week 17 data

In [ ]:
# Filter players by position
qbs = offense_week17[offense_week17['Position'] == 'QB']
rbs = offense_week17[offense_week17['Position'] == 'RB']
wrs = offense_week17[offense_week17['Position'] == 'WR']
tes = offense_week17[offense_week17['Position'] == 'TE']
flex_players = pd.concat([rbs, wrs, tes])
flex_players['Position'] = 'FLEX'  # Add a 'Position' column to identify flex players
dsts = defense_week17

In [ ]:
# Create a new optimization model for week 17
model_week17 = pe.ConcreteModel()

# Define decision variables (similar to the previous model)
model_week17.qb_vars = pe.Var(qbs.index, within=pe.Binary)
model_week17.rb_vars = pe.Var(rbs.index, within=pe.Binary)
model_week17.wr_vars = pe.Var(wrs.index, within=pe.Binary)
model_week17.te_vars = pe.Var(tes.index, within=pe.Binary)
model_week17.flex_vars = pe.Var(flex_players.index, within=pe.Binary)
model_week17.dst_vars = pe.Var(dsts.index, within=pe.Binary)

# Define the objective function for week 17
def objective_rule_week17(model):
    return sum(model.qb_vars[i] * qbs.loc[i, 'FantasyPoints'] for i in qbs.index) + \
           sum(model.rb_vars[i] * rbs.loc[i, 'FantasyPoints'] for i in rbs.index) + \
           sum(model.wr_vars[i] * wrs.loc[i, 'FantasyPoints'] for i in wrs.index) + \
           sum(model.te_vars[i] * tes.loc[i, 'FantasyPoints'] for i in tes.index) + \
           sum(model.flex_vars[i] * flex_players.loc[i, 'FantasyPoints'] for i in flex_players.index) + \
           sum(model.dst_vars[i] * dsts.loc[i, 'AvgPointsPerGame'] for i in dsts.index)

model_week17.objective = pe.Objective(rule=objective_rule_week17, sense=pe.maximize)

Constraints (again)

In [ ]:
# Constraint: Each player can only be selected once (modified for FLEX)
def player_uniqueness_rule(model_week17):
    all_players = set(qbs['Player']).union(set(rbs['Player']), set(wrs['Player']), set(tes['Player']))
    for player in all_players:
        # Count selections across all positions, including FLEX
        selected_count = sum(model_week17.qb_vars[i] for i in qbs.index if qbs.loc[i, 'Player'] == player) + \
                         sum(model_week17.rb_vars[i] for i in rbs.index if rbs.loc[i, 'Player'] == player) + \
                         sum(model_week17.wr_vars[i] for i in wrs.index if wrs.loc[i, 'Player'] == player) + \
                         sum(model_week17.te_vars[i] for i in tes.index if tes.loc[i, 'Player'] == player) + \
                         sum(model_week17.flex_vars[i] for i in flex_players.index if flex_players.loc[i, 'Player'] == player)

        # Constraint for each player to be selected at most once
        model_week17.add_component(f'player_uniqueness_constraint_{player}', pe.Constraint(expr=selected_count <= 1))

    # Defense
    for player in set(dsts['Player']):
        model_week17.add_component(f'player_uniqueness_constraint_dst_{player}', pe.Constraint(expr=sum(model_week17.dst_vars[i] for i in dsts.index if dsts.loc[i, 'Player'] == player) <= 1))

    return Constraint.Feasible

model_week17.player_uniqueness_constraint = pe.Constraint(rule=player_uniqueness_rule)

In [ ]:
# Define constraints for the number of players at each position
model_week17.qbs_constraint = pe.Constraint(expr=sum(model_week17.qb_vars[i] for i in qbs.index) == 1)
model_week17.rbs_constraint = pe.Constraint(expr=sum(model_week17.rb_vars[i] for i in rbs.index) == 2)
model_week17.wrs_constraint = pe.Constraint(expr=sum(model_week17.wr_vars[i] for i in wrs.index) == 3)
model_week17.tes_constraint = pe.Constraint(expr=sum(model_week17.te_vars[i] for i in tes.index) == 1)
model_week17.flex_constraint = pe.Constraint(expr=sum(model_week17.flex_vars[i] for i in flex_players.index) == 1)
model_week17.dsts_constraint = pe.Constraint(expr=sum(model_week17.dst_vars[i] for i in dsts.index) == 1)

#Total players constraint
model_week17.total_players = pe.Constraint(expr=sum(model_week17.qb_vars[i] for i in qbs.index) +
                                     sum(model_week17.rb_vars[i] for i in rbs.index) +
                                     sum(model_week17.wr_vars[i] for i in wrs.index) +
                                     sum(model_week17.te_vars[i] for i in tes.index) +
                                     sum(model_week17.flex_vars[i] for i in flex_players.index) +
                                     sum(model_week17.dst_vars[i] for i in dsts.index) == 9)


In [ ]:
# Define the salary cap constraint
model_week17.salary_cap = pe.Constraint(expr=sum(model_week17.qb_vars[i] * qbs.loc[i, 'Salary'] for i in qbs.index) +
                                     sum(model_week17.rb_vars[i] * rbs.loc[i, 'Salary'] for i in rbs.index) +
                                     sum(model_week17.wr_vars[i] * wrs.loc[i, 'Salary'] for i in wrs.index) +
                                     sum(model_week17.te_vars[i] * tes.loc[i, 'Salary'] for i in tes.index) +
                                     sum(model_week17.flex_vars[i] * flex_players.loc[i, 'Salary'] for i in flex_players.index) +
                                     sum(model_week17.dst_vars[i] * dsts.loc[i, 'Salary'] for i in dsts.index) <= 50000)

In [ ]:
# Create a set of unique game IDs from the 'Game Info' column
games = set()
for index, row in offense_week17.iterrows():
    games.add(row['Game Info'])
for index, row in defense_week17.iterrows():
    games.add(row['Game Info'])
game_list = list(games)
game_index = range(len(game_list))
game_map = dict(zip(game_list, game_index))


# Constraint: At least 2 different games represented
model_week17.game_constraint = pe.Constraint(expr=sum(model_week17.qb_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in qbs.index) +
                                        sum(model_week17.rb_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in rbs.index) +
                                        sum(model_week17.wr_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in wrs.index) +
                                        sum(model_week17.te_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in tes.index) +
                                        sum(model_week17.flex_vars[i] * game_map[offense_season.loc[i, 'Game Info']] for i in flex_players.index) +
                                        sum(model_week17.dst_vars[i] * game_map[defense_season.loc[i, 'Game Info']] for i in dsts.index) >= 2 )


My constraints

In [ ]:
offense_week17.columns

Index(['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary',
       'Game Info', 'TeamAbbrev', 'AvgPointsPerGame', 'Player', 'Tm',
       'Pass Cmp', 'Pass Att', 'Pass Yds', 'Pass TD', 'Pass Int', 'Pass Sack',
       'Pass Sack Yds', 'Pass Long Complete', 'Ruch Att', 'Rush Yds',
       'Rush TD', 'Rush Lng', 'Rcv Tgt', 'Rec', 'Rcv Yds', 'Rcv TD', 'Rcv Lng',
       'Fmb', 'FL', '2Pt Conversions', 'Ttl Pts Allowed', 'Safety',
       'Blocked Kicks', 'Sack', 'Interception', 'Fumble Recovery',
       'Punt/Kickoff/FG Return for TD', 'Interception Return TD',
       'Fumble Recovery TD', 'Blocked Punt or FG Return TD', 'FantasyPoints'],
      dtype='object')

In [ ]:
# Calculate interception percentage and adjusted fantasy points
# Updated to use the correct column names and calculations
offense_week17['Int%'] = offense_week17['Pass Int'] / offense_week17['Pass Att']  # Using 'Pass Int' and 'Pass Att'

# Replace any infinite values (from division by 0) with 0
offense_week17['Int%'].replace([np.inf, -np.inf], 0, inplace=True)
offense_week17['Int%'].fillna(0, inplace=True)  # Replace any remaining NaNs with 0

# Calculate adjusted fantasy points
offense_week17['AdjustedFantasyPoints'] = offense_week17['FantasyPoints'] * (1 - (offense_week17['Int%'] / 10))

# Identify the top 10% of QBs by interception rate
top_10_percent_int_qbs = offense_week17[offense_week17['Position'] == 'QB'].nlargest(int(0.1 * len(offense_week17[offense_week17['Position'] == 'QB'])), 'Int%')

# Constraint: Exclude QBs in the top 10% of interception rates
model_week17.interception_constraint = pe.Constraint(expr=sum(model_week17.qb_vars[i] for i in top_10_percent_int_qbs.index) == 0)

<ipython-input-3345-ce98ca0d7cfd>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  offense_week17['Int%'].replace([np.inf, -np.inf], 0, inplace=True)
<ipython-input-3345-ce98ca0d7cfd>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

changed to work with TeamAbbrev column instead of team column

In [ ]:
# Constraint: If a QB is selected, at least one WR or TE from the same team must also be selected
model_week17.qb_wr_te_stacking = pe.ConstraintList()

for team_abbrev in offense_week17['TeamAbbrev'].unique():
    qb_indices = [i for i in qbs.index if qbs.loc[i, 'TeamAbbrev'] == team_abbrev]
    wr_te_indices = [i for i in wrs.index if wrs.loc[i, 'TeamAbbrev'] == team_abbrev] + \
                    [i for i in tes.index if tes.loc[i, 'TeamAbbrev'] == team_abbrev]

    # Only apply the constraint if both QBs and WR/TEs exist for this team abbreviation
    if qb_indices and wr_te_indices:
        qb_selected = sum(model_week17.qb_vars[i] for i in qb_indices)
        wr_te_selected = sum(model_week17.wr_vars[i] for i in wr_te_indices if i in wrs.index) + \
                         sum(model_week17.te_vars[i] for i in wr_te_indices if i in tes.index)

        # Add the constraint
        model_week17.qb_wr_te_stacking.add(qb_selected <= wr_te_selected)


In [ ]:
# Define a threshold for "consistent" players
consistency_threshold = 10  # Players must average at least 10 fantasy points per game

# Identify players who meet this threshold
consistent_players = offense_week17[offense_season['AvgPointsPerGame'] >= consistency_threshold].index.tolist()

<ipython-input-3347-ea16db53e704>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  consistent_players = offense_week17[offense_season['AvgPointsPerGame'] >= consistency_threshold].index.tolist()


In [ ]:
# Constraint: At least 4 selected players must have AvgPointsPerGame >= consistency_threshold
def high_floor_player_selection_rule(x):
    consistent_players_selected = (
        sum(model_week17.qb_vars[i] for i in qbs.index if i in consistent_players) +
        sum(model_week17.rb_vars[i] for i in rbs.index if i in consistent_players) +
        sum(model_week17.wr_vars[i] for i in wrs.index if i in consistent_players) +
        sum(model_week17.te_vars[i] for i in tes.index if i in consistent_players) +
        sum(model_week17.flex_vars[i] for i in flex_players.index if i in consistent_players)
    )
    return consistent_players_selected >= 4  # Require at least 4 consistent players

model_week17.high_floor_player_selection = pe.Constraint(rule=high_floor_player_selection_rule)

In [ ]:
# Solve the week 17 model
opt = SolverFactory('appsi_highs')
results_week17 = opt.solve(model_week17)


In [ ]:
# Extract the chosen players from the solution for week 17
chosen_players_week17 = []
for i in qbs.index:
    if model_week17.qb_vars[i].value == 1:
        chosen_players_week17.append((qbs.loc[i, 'Player'], 'QB'))
for i in rbs.index:
    if model_week17.rb_vars[i].value == 1:
        chosen_players_week17.append((rbs.loc[i, 'Player'], 'RB'))
for i in wrs.index:
    if model_week17.wr_vars[i].value == 1:
        chosen_players_week17.append((wrs.loc[i, 'Player'], 'WR'))
for i in tes.index:
    if model_week17.te_vars[i].value == 1:
        chosen_players_week17.append((tes.loc[i, 'Player'], 'TE'))
for i in flex_players.index:
    if model_week17.flex_vars[i].value == 1:
        chosen_players_week17.append((flex_players.loc[i, 'Player'], flex_players.loc[i, 'Position']))
for i in dsts.index:
    if model_week17.dst_vars[i].value == 1:
        chosen_players_week17.append((dsts.loc[i, 'TeamAbbrev'], 'DST'))

# Print the chosen players for week 17
for player, position in chosen_players_week17:
    print(f"{player} ({position})")


drewlock (QB)
jonathantaylor (RB)
buckyirving (RB)
teehiggins (WR)
maliknabers (WR)
rickypearsall (WR)
treymcbride (TE)
zachertz (FLEX)
DEN (DST)


In [ ]:
# Access the objective function value (maximum fantasy points + average points scored)
max_fantasy_points = model_week17.objective()

print(f"Maximum Projected Fantasy Points: {max_fantasy_points}")

Maximum Projected Fantasy Points: 277.89


In [ ]:
#print out the total slary used in the model_week17

total_salary_week17 = sum(model_week17.qb_vars[i].value * qbs.loc[i, 'Salary'] for i in qbs.index) + \
                      sum(model_week17.rb_vars[i].value * rbs.loc[i, 'Salary'] for i in rbs.index) + \
                      sum(model_week17.wr_vars[i].value * wrs.loc[i, 'Salary'] for i in wrs.index) + \
                      sum(model_week17.te_vars[i].value * tes.loc[i, 'Salary'] for i in tes.index) + \
                      sum(model_week17.flex_vars[i].value * flex_players.loc[i, 'Salary'] for i in flex_players.index) + \
                      sum(model_week17.dst_vars[i].value * dsts.loc[i, 'Salary'] for i in dsts.index)

print(f"Total Salary Used in model_week17: {total_salary_week17}")


Total Salary Used in model_week17: 50000.0


#Yay we did it! We answered both questions using all our constraints and following all the rules! Now go pick your fantasy football team!